# Data Preparation

In [42]:
import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

from pprint import pprint


 This study is based on a survey conducted on 103904 airline passengers. There is limited information on the background of the dataset and how the data was collected. The dataset has been downloaded from the Kaggle website and can be found at the following [link](#ref-scikitdevelopers2024). The project's main goal is to determine the features most strongly correlated with customer satisfaction and to create a model to predict new customer satisfaction levels. Since the response variable, 'satisfaction', is binary, it is a classification problem or a supervised learning task. There are 23 features in total, most of which are categorical in nature. According to the authors of the data's website', it has already been preprocessed for the purpose of classification. However, the dataset will be validated and reprocessed in this notebook.  

## Cleaning and Transforming the Train Data Set

In [43]:
train = pd.read_csv('../Data/Raw/train.csv', index_col = 0)
train.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


A quick check for missing values reveals that one feature does contain missing values. Other data parameters are also checked below.

In [44]:
train.isna().any()

id                                   False
Gender                               False
Customer Type                        False
Age                                  False
Type of Travel                       False
Class                                False
Flight Distance                      False
Inflight wifi service                False
Departure/Arrival time convenient    False
Ease of Online booking               False
Gate location                        False
Food and drink                       False
Online boarding                      False
Seat comfort                         False
Inflight entertainment               False
On-board service                     False
Leg room service                     False
Baggage handling                     False
Checkin service                      False
Inflight service                     False
Cleanliness                          False
Departure Delay in Minutes           False
Arrival Delay in Minutes              True
satisfactio

In [45]:
# Checking the total amount of missing values:
print('Total missing values in the data:', train['Arrival Delay in Minutes'].isna().sum())

Total missing values in the data: 310


In [46]:
# Having a quick look at the data types:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     103904 no

In [47]:
# Checking for duplicate entries:
print('Data is duplicated:', train.duplicated().any())

Data is duplicated: False


In [48]:
# Checking the shape of the dataset:
print('Data shape:', train.shape)

Data shape: (103904, 24)


Each feature in the dataset will be evaluated separately. This is the first step in familiarising oneself with the data to ensure every variable is correctly encoded.

In [49]:
# Checking whether the `id` column has only unique values:
print('Length of unique ID values:', len(train['id'].unique()))
print('Total entries in the dataset:', len(train['id']))

Length of unique ID values: 103904
Total entries in the dataset: 103904


In [50]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Gender'].unique())

# Coverting to categorical:
train['Gender'] = train['Gender'].astype('category')
print('\n\033[1mProcessed values:\033[0m',train['Gender'].unique())

Original values: ['Male' 'Female']

Processed values: ['Male', 'Female']
Categories (2, object): ['Female', 'Male']


In [51]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Customer Type'].unique())

# Formatting the column:
train['Customer Type'] = train['Customer Type'].str.capitalize()

# Coverting to categorical:
train['Customer Type'] = train['Customer Type'].astype('category')
print('\n\033[1mProcessed values:\033[0m', train['Customer Type'].unique())

Original values: ['Loyal Customer' 'disloyal Customer']

Processed values: ['Loyal customer', 'Disloyal customer']
Categories (2, object): ['Disloyal customer', 'Loyal customer']


In [52]:
# Unique column values:
print('\033[1mUnique values:\033[0m', train['Age'].unique())

# Validating the data:
print('\n\033[1mMinimum Age:\033[0m', train['Age'].min())
print('\n\033[1mMaximum Age:\033[0m', train['Age'].max())

Unique values: [13 25 26 61 47 52 41 20 24 12 53 33 45 38  9 17 43 58 23 57 49 36 22 31
 15 35 67 37 40 34 39 50 29 54 21 28 27 69 60 48 59 46 30 66 64 44 51 32
 19 42 16 11 62  8 56 68 55 18 65 72 70 63 10  7 14 80 74 71 85 73 76 77
 75 79 78]

Minimum Age: 7

Maximum Age: 85


In [53]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Type of Travel'].unique())

# Capitalising the info:
train['Type of Travel'] = train['Type of Travel'].str.capitalize()

# Coverting to categorical:
train['Type of Travel'] = train['Type of Travel'].astype('category')
print('\n\033[1mProcessed values:\033[0m', train['Type of Travel'].unique())

Original values: ['Personal Travel' 'Business travel']

Processed values: ['Personal travel', 'Business travel']
Categories (2, object): ['Business travel', 'Personal travel']


In [54]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Class'].unique())
train['Class'] = train['Class'].str.replace('Eco', '1')
train['Class'] = train['Class'].str.replace('1 Plus', '2')
train['Class'] = train['Class'].str.replace('Business', '3')

# Converting to categorical:
train['Class'] = train['Class'].astype('category')
train['Class'] = train['Class'].cat.set_categories(new_categories = ['1', '2', '3'], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Class'].unique())

Original values: ['Eco Plus' 'Business' 'Eco']

Processed values: ['2', '3', '1']
Categories (3, object): ['1' < '2' < '3']


In [55]:
# Unique column values:
print('\033[1mUnique values:\033[0m', train['Flight Distance'].unique())

# Validating the data:
print('\n\033[1mMinimum Flight Distance:\033[0m', train['Flight Distance'].min())
print('\n\033[1mMaximum Flight Distance:\033[0m', train['Flight Distance'].max())

Unique values: [ 460  235 1142 ...  974 1479  400]

Minimum Flight Distance: 31

Maximum Flight Distance: 4983


In [56]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Inflight wifi service'].unique())

# Converting to ordered categorical:
train['Inflight wifi service'] = train['Inflight wifi service'].astype('category')
train['Inflight wifi service'] = train['Inflight wifi service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Inflight wifi service'].unique())

Original values: [3 2 4 1 5 0]

Processed values: [3, 2, 4, 1, 5, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [57]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Departure/Arrival time convenient'].unique())

# Converting to ordered categorical:
train['Departure/Arrival time convenient'] = train['Departure/Arrival time convenient'].astype('category')
# Rearranging category order:
train['Departure/Arrival time convenient'] = train['Departure/Arrival time convenient'].cat.set_categories(new_categories = [1, 2, 3, 4, 5],
                                                                                                           ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Departure/Arrival time convenient'].unique())


Original values: [4 2 5 3 1 0]

Processed values: [4, 2, 5, 3, 1, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [58]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Ease of Online booking'].unique())

# Converting to ordered categorical:
train['Ease of Online booking'] = train['Ease of Online booking'].astype('category')
train['Ease of Online booking'] = train['Ease of Online booking'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], 
                                                                                     ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Ease of Online booking'].unique())

Original values: [3 2 5 4 1 0]

Processed values: [3, 2, 5, 4, 1, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [59]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Gate location'].unique())

# Converting to ordered categorical:
train['Gate location'] = train['Gate location'].astype('category')
train['Gate location'] = train['Gate location'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Gate location'].unique())

Original values: [1 3 2 5 4 0]

Processed values: [1, 3, 2, 5, 4, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [60]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Food and drink'].unique())

# Converting to ordered categorical:
train['Food and drink'] = train['Food and drink'].astype('category')
train['Food and drink'] = train['Food and drink'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Food and drink'].unique())

Original values: [5 1 2 4 3 0]

Processed values: [5, 1, 2, 4, 3, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [61]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Online boarding'].unique())

# Converting to ordered categorical:
train['Online boarding'] = train['Online boarding'].astype('category')
train['Online boarding'] = train['Online boarding'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Online boarding'].unique())

Original values: [3 5 2 1 4 0]

Processed values: [3, 5, 2, 1, 4, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [62]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Seat comfort'].unique())

# Converting to ordered categorical:
train['Seat comfort'] = train['Seat comfort'].astype('category')
train['Seat comfort'] = train['Seat comfort'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Seat comfort'].unique())


Original values: [5 1 2 3 4 0]

Processed values: [5, 1, 2, 3, 4, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [63]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Inflight entertainment'].unique())

# Converting to ordered categorical:
train['Inflight entertainment'] = train['Inflight entertainment'].astype('category')
train['Inflight entertainment'] = train['Inflight entertainment'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Inflight entertainment'].unique())

Original values: [5 1 2 3 4 0]

Processed values: [5, 1, 2, 3, 4, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [64]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['On-board service'].unique())

# Converting to ordered categorical:
train['On-board service'] = train['On-board service'].astype('category')
train['On-board service'] = train['On-board service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['On-board service'].unique())


Original values: [4 1 2 3 5 0]

Processed values: [4, 1, 2, 3, 5, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [65]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Leg room service'].unique())

# Converting to ordered categorical:
train['Leg room service'] = train['Leg room service'].astype('category')
train['Leg room service'] = train['Leg room service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Leg room service'].unique())

Original values: [3 5 4 2 1 0]

Processed values: [3, 5, 4, 2, 1, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [66]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Baggage handling'].unique())

# Converting to ordered categorical:
train['Baggage handling'] = train['Baggage handling'].astype('category')
train['Baggage handling'] = train['Baggage handling'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Baggage handling'].unique())


Original values: [4 3 5 1 2]

Processed values: [4, 3, 5, 1, 2]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [67]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Checkin service'].unique())

# Converting to ordered categorical:
train['Checkin service'] = train['Checkin service'].astype('category')
train['Checkin service'] = train['Checkin service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Checkin service'].unique())

Original values: [4 1 3 5 2 0]

Processed values: [4, 1, 3, 5, 2, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [68]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Inflight service'].unique())

# Converting to cordered ategorical:
train['Inflight service'] = train['Inflight service'].astype('category')
train['Inflight service'] = train['Inflight service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Inflight service'].unique())

Original values: [5 4 3 1 2 0]

Processed values: [5, 4, 3, 1, 2, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [69]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['Cleanliness'].unique())

# Converting to ordered categorical:
train['Cleanliness'] = train['Cleanliness'].astype('category')
train['Cleanliness'] = train['Cleanliness'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)
print('\n\033[1mProcessed values:\033[0m', train['Cleanliness'].unique())

Original values: [5 1 2 3 4 0]

Processed values: [5, 1, 2, 3, 4, NaN]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]


In [70]:
# Unique column values:
print('\033[1mUnique values:\033[0m', train['Departure Delay in Minutes'].unique()[0:50])

# Checking column dtype:
print('\n\033[1mDtype:\033[0m', train['Departure Delay in Minutes'].dtype)

# Validating data:
print('\n\033[1mMinimum Departure Delay in Minutes:\033[0m', train['Departure Delay in Minutes'].min())
print('\n\033[1mMaximum Departure Delay in Minutes:\033[0m', train['Departure Delay in Minutes'].max())

Unique values: [ 25   1   0  11   9   4  28  43  49   7  17  52  54  27  18  19   3 109
  23   8  14  10  51  39  13  30  64  20  45  44  31  81  35  67  22  40
  91  21  15  29 105  12 162  24 141   6  34   2  97  16]

Dtype: int64

Minimum Departure Delay in Minutes: 0

Maximum Departure Delay in Minutes: 1592


In [71]:
# Unique column values:
print('\033[1mUnique values:\033[0m', train['Arrival Delay in Minutes'].unique()[0:100])

# Checking column dtype:
print('\n\033[1mDtype:\033[0m', train['Arrival Delay in Minutes'].dtype)

# Validating data:
print('\n\033[1mMinimum Departure Delay in Minutes:\033[0m', train['Arrival Delay in Minutes'].min())
print('\n\033[1mMaximum Departure Delay in Minutes:\033[0m', train['Arrival Delay in Minutes'].max())

Unique values: [ 18.   6.   0.   9.  23.   8.  35.  51.  10.   5.   4.  29.  44.  28.
  12. 120.  24.   1.  20.  31.  15.  48.  26.  49.   2.  37.  50.   3.
  19.  72.  11.  34.  62.  27.  52.  13.  82.  30.  16.   7. 122. 179.
 125.  17.  nan  89. 101.  14.  61.  32.  33.  41. 191. 138.  53.  22.
  57.  65.  76. 107.  92. 164.  21.  40.  55. 185.  63.  77.  86.  91.
 100.  54.  36.  70. 139.  67. 163. 128. 180.  93. 121.  45. 105. 126.
  56.  73. 212.  88. 241. 172. 175. 111.  99.  25.  42. 226.  46. 131.
 260.  69.]

Dtype: float64

Minimum Departure Delay in Minutes: 0.0

Maximum Departure Delay in Minutes: 1584.0


In [72]:
# Unique column values:
print('\033[1mOriginal values:\033[0m', train['satisfaction'].unique())

# Formatting column name:
train = train.rename(columns = {'satisfaction': 'Satisfaction'})
train['Satisfaction'].unique()

# Converting to ordered categorical:
train['Satisfaction'] = train['Satisfaction'].astype('category')
train['Satisfaction'] = train['Satisfaction'].cat.set_categories(new_categories = ['neutral or dissatisfied', 'satisfied'], ordered = True)

# Unique column values:
print('\n\033[1mProcessed values:\033[0m', train['Satisfaction'].unique())

Original values: ['neutral or dissatisfied' 'satisfied']

Processed values: ['neutral or dissatisfied', 'satisfied']
Categories (2, object): ['neutral or dissatisfied' < 'satisfied']


In [73]:
# Formating column name:
train = train.rename(columns = {'Inflight wifi service': 'Inflight Wifi Service', 'Departure/Arrival time convenient': 'Departure/Arrival Time Convenient',
                                'Ease of Online booking': 'Ease of Online Booking', 'Gate location': 'Gate Location', 
                                'Food and drink': 'Food and Drink', 'Online boarding': 'Online Boarding', 'Seat comfort': 'Seat Comfort',
                                'Inflight entertainment': 'Inflight Entertainment', 'On-board service': 'On-board Service', 'Leg room service': 'Leg Room Service',
                                'Baggage handling': 'Baggage Handling', 'Checkin service': 'Checkin Service', 'Inflight service': 'Inflight Service'})


Notice how many NaNs were created when cleaning the data and setting correct ordered category values. This will be further analysed below:

In [74]:
# Checking for missing values:
train.isna().sum()

id                                      0
Gender                                  0
Customer Type                           0
Age                                     0
Type of Travel                          0
Class                                   0
Flight Distance                         0
Inflight Wifi Service                3103
Departure/Arrival Time Convenient    5300
Ease of Online Booking               4487
Gate Location                           1
Food and Drink                        107
Online Boarding                      2428
Seat Comfort                            1
Inflight Entertainment                 14
On-board Service                        3
Leg Room Service                      472
Baggage Handling                        0
Checkin Service                         1
Inflight Service                        3
Cleanliness                            12
Departure Delay in Minutes              0
Arrival Delay in Minutes              310
Satisfaction                      

Prior to further analysis, this version of the dataset will be stored for use in the `exploratory_data_analysis` notebook.

In [1901]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train.to_pickle('../Data/Preprocessed/train_preprocessed.pkl')

Additionally, the response variable must be formatted in a numeric format. Using a dictionary, the 'neutral or dissatisfied' category is converted to '1', and the 'satisfied' category is converted to '2'. Since 'neutral or dissatisfied' is less than 'satisfied', the category is set to ‘ordered’. 

In [76]:
# Defining the variable transformation needed:
change = {'satisfied': 2, 'neutral or dissatisfied': 1}

# Performing the transformation and converting to integer:
train['Satisfaction'] = train['Satisfaction'].replace(change).infer_objects(copy=False)
train['Satisfaction'] = train['Satisfaction'].astype('int')

# Converting to ordered categorical:
train['Satisfaction'] = train['Satisfaction'].astype('category')
train['Satisfaction'] = train['Satisfaction'].cat.set_categories(new_categories = [1, 2], ordered = True)

/var/folders/1z/2qtxqpxj07g0pvc7h463fm_w0000gn/T/ipykernel_92608/1028444235.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['Satisfaction'] = train['Satisfaction'].replace(change).infer_objects(copy=False)
/var/folders/1z/2qtxqpxj07g0pvc7h463fm_w0000gn/T/ipykernel_92608/1028444235.py:5: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  train['Satisfaction'] = train['Satisfaction'].replace(change).infer_objects(copy=False)


This version of the dataset will also be saved for use in the `exploratory_data_analysis` notebook.

In [1903]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train.to_pickle('../Data/Preprocessed/train_preprocessed_2.pkl')

To implement machine learning models, nominal categorical variables must also be formatted into numerical. The ideal way to do this is through Panda's `get_dummies` function. This function creates extra columns with dummy variables corresponding to the total number of categories in the variables of interest. 

In [78]:
# Getting dummies for the nominal variables:
train = pd.get_dummies(train, columns = ['Gender', 'Customer Type', 'Type of Travel'], prefix = 'Dummy', dtype = int)

In [1905]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train.to_pickle('../Data/Preprocessed/train_preprocessed_3.pkl')

## Dealing with NaNs

The dataset description mentions that many of the categorical variables have a 'Not Applicable' category. By taking a closer look at these values (see below), it can be seen that often when a feature has NaNs, other features also have NaNs. However, the pattern is inconsistent; for example, the dataset below has been filtered for NaN values in `Ease of Online Booking`. For these entries, the values of `Departure/Arrival Time Convenient` and `Inflight Wifi Service` are only sometimes missing. For this reason, it can be concluded that the data seems to be **Missing At Random** (MAR). 

This is quite a common problem in survey data. Many survey respondents may skip over questions because they did not use the service; others simply did not understand the question and preferred to mark the answer as 'Not Applicable'. The latter could easily apply to elder individuals [Hannah Igboke, 2024](#ref-Igkobe2024).

If the data is missing, then a good strategy would be to use imputation methods to 'extrapolate' the missing values from the available data. If, on the other hand, the data is actually 'Not Applicable’, then dropping these entries or accepting them as such might be a better strategy. Both situations will be evaluated.

In [1906]:
train[train['Ease of Online Booking'].isna() == True]

,id,Age,Class,Flight Distance,Inflight Wifi Service,Departure/Arrival Time Convenient,Ease of Online Booking,Gate Location,Food and Drink,Online Boarding,...,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Satisfaction,Dummy_Female,Dummy_Male,Dummy_Disloyal customer,Dummy_Loyal customer,Dummy_Business travel,Dummy_Personal travel
42,14849,41,3,296,NaN,NaN,NaN,3,2,5,...,3,0,0.0,2,0,1,0,1,1,0
56,78972,57,3,2496,NaN,NaN,NaN,1,3,4,...,5,0,5.0,2,1,0,0,1,1,0
90,76392,60,1,931,NaN,5,NaN,3,4,NaN,...,4,0,0.0,2,0,1,0,1,0,1
148,73604,55,3,3634,NaN,4,NaN,4,4,5,...,3,0,0.0,2,1,0,0,1,1,0
162,104623,20,1,861,NaN,NaN,NaN,5,2,NaN,...,2,0,0.0,2,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103683,17529,24,2,241,NaN,2,NaN,3,2,NaN,...,2,1,0.0,2,1,0,1,0,1,0
103700,50815,24,1,266,2,NaN,NaN,5,1,NaN,...,1,0,0.0,1,0,1,0,1,0,1
103728,15861,49,3,3578,NaN,NaN,NaN,3,2,1,...,3,65,95.0,2,1,0,0,1,1,0
103814,70449,64,1,187,2,4,NaN,2,3,NaN,...,3,0,0.0,1,0,1,0,1,0,1


### Iterative Imputer

Scikit-learn's IterativeImputer runs a regression for each variable with a ‘Not Applicable’ categorical entry and uses all other features to impute the ‘missing’ value. The model is then trained on the new data, and the process is repeated several times [(Scikit-learn developers, 2024)](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html).

In [1907]:
# Downloading the saved file: 
train = pd.read_pickle('../Data/Preprocessed/train_preprocessed_3.pkl')

Next, the IterativeImputer will be instantiated. Since one numeric feature has missing values, the `initial_strategy` will be set to the 'mean'. A `random_state` will also be set to ensure reproducibility. The `max_iter` parameter is increased to 20 (the default is 10) to obtain good results. The instantiated imputer is then used to impute the dataset's 'missing' values. 

In [1908]:
# Instantiating the IterativeImputer:
imputer = IterativeImputer(initial_strategy='mean', random_state = 42, max_iter = 20)

# Imputing the data with the instantiated imputer:
imputed_data = imputer.fit_transform(train)
train_imputed_iterative = pd.DataFrame(imputed_data, columns = train.columns)


The resulting dataset is saved for further evaluation in the `model_training_and_evaluation` notebook. 

In [1909]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train_imputed_iterative.to_pickle('../Data/Preprocessed/train_imputed_iterative.pkl')

### KNN Imputer

The KNN imputation method will also be attempted. This method imputes the feature's missing values based on the *k*-nearest neighbours using the whole dataset. The neighbours are determined using Euclidean distance by default. This method requires all numerical values to be scaled in order to avoid imputation bias [Kyaw Saw Htoon, 2020](#ref-htoon2020). 

In [1910]:
# Importing the first preprocessed dataset:
train = pd.read_pickle('../Data/Preprocessed/train_preprocessed_3.pkl')

Here, numerical data will be normalised using Scikit-learns' MinMaxScaler. 

In [1911]:
scaler = MinMaxScaler()

train = pd.DataFrame(scaler.fit_transform(train), columns = train.columns)

Next, the KNN imputer is instantiated with 18000 neighbours. Although it is common practice to set the `n_neighbors` parameter of the KNNimputer to the square root of the total number of observations, `n_neighbors = sqrt(n_obs)`, that is $k = 322$. However, after some manual hyperparameter tuning, a larger value of $k$ provides better results. This makes sense since the dataset is quite large.

In [1912]:
# Instantiating the KNN imputer:
imputer_knn = KNNImputer(n_neighbors=18000)

# Imputing the missing values:
imputed_data_knn = imputer_knn.fit_transform(train)
train_imputed_knn = pd.DataFrame(imputed_data_knn, columns = train.columns)

Saving the new data set for further analysis in the `model_training_and_evaluation` notebook. 

In [1913]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train_imputed_knn.to_pickle('../Data/Preprocessed/train_imputed_knn.pkl')

### Dropna with Mode Imputation

In [1914]:
train = pd.read_pickle('../Data/Preprocessed/train_preprocessed_3.pkl')

In this section, the entries with 'Non Applicable' categories are considered meaningless for the analysis and are dropped. Entries where NaN values are less than 5% of the data will be dropped. 

In [1915]:
# Establishing a threshold in accordance with the dataset size for the deletion of NaNs:
threshold = len(train)*0.05
print('Threshold for dropping NaNs:', round(threshold))

# Identifying the columns for which NaNs can be dropped:
columns_to_drop = train.columns[train.isna().sum() <= threshold]
print('\nColumns where NaN entries can be dropped:')
pprint(list(columns_to_drop))

# Identifying the columns for which NaNs should be imputed:
cols_to_impute = train.columns[train.isna().sum() > threshold]
print('\nColumns where NaNs need to be imputed:')
pprint(list(cols_to_impute))

# Dropping columns with small amounts of NaNs:
train = train.dropna(subset=columns_to_drop)

Threshold for dropping NaNs: 5195

Columns where NaN entries can be dropped:
['id',
 'Age',
 'Class',
 'Flight Distance',
 'Inflight Wifi Service',
 'Ease of Online Booking',
 'Gate Location',
 'Food and Drink',
 'Online Boarding',
 'Seat Comfort',
 'Inflight Entertainment',
 'On-board Service',
 'Leg Room Service',
 'Baggage Handling',
 'Checkin Service',
 'Inflight Service',
 'Cleanliness',
 'Departure Delay in Minutes',
 'Arrival Delay in Minutes',
 'Satisfaction',
 'Dummy_Female',
 'Dummy_Male',
 'Dummy_Disloyal customer',
 'Dummy_Loyal customer',
 'Dummy_Business travel',
 'Dummy_Personal travel']

Columns where NaNs need to be imputed:
['Departure/Arrival Time Convenient']


Notice how the set of features where NaNs can be dropped includes the numeric variable 'Arrival Delay in Minutes'. For all other columns where there are more missing values than the established threshold the NaNs will be imputed using the feature's mode. 

In [1916]:
# Iterate through each column to impute NaNs with the mode:
for col in cols_to_impute:
    mode_value = train[col].mode()
    if not mode_value.empty:
        train[col] = train[col].fillna(mode_value.iloc[0])

Saving the new data set for further analysis in the `model_training_and_evaluation` notebook. 

In [1917]:
# Saving the first preprocessed dataset to pickle to preserve data type information:
# train.to_pickle('../Data/Preprocessed/train_preprocessed_dropna_mode.pkl')

### Creating an Extra Category for 'Non-Applicable'

In [8]:
train = pd.read_pickle('../Data/Preprocessed/train_preprocessed_3.pkl')

The last attempt at dealing with the NaN values will be to create a separate numeric category for such entries.

In [1919]:
# Defining the categorical features with NaN values:
non_applicable_category = ['Inflight Wifi Service', 'Departure/Arrival Time Convenient', 'Ease of Online Booking', 'Gate Location', 'Food and Drink', 'Online Boarding',
                           'Seat Comfort', 'Inflight Entertainment', 'On-board Service', 'Leg Room Service', 'Checkin Service', 'Inflight Service', 'Cleanliness']

# Setting new category values for the defined features:
for feature in non_applicable_category:
    train[feature] = train[feature].astype('category')
    train[feature] = train[feature].cat.set_categories(new_categories = [0, 1, 2, 3, 4, 5], ordered = True)

# Replacing the NaNs with 0:
assigning = {np.nan: 0}

for feature in non_applicable_category:
    train[feature] = train[feature].fillna(0)

In [1920]:
# Checking for the remaining missing values:
train.isna().any()

id                                   False
Age                                  False
Class                                False
Flight Distance                      False
Inflight Wifi Service                False
Departure/Arrival Time Convenient    False
Ease of Online Booking               False
Gate Location                        False
Food and Drink                       False
Online Boarding                      False
Seat Comfort                         False
Inflight Entertainment               False
On-board Service                     False
Leg Room Service                     False
Baggage Handling                     False
Checkin Service                      False
Inflight Service                     False
Cleanliness                          False
Departure Delay in Minutes           False
Arrival Delay in Minutes              True
Satisfaction                         False
Dummy_Female                         False
Dummy_Male                           False
Dummy_Dislo

The last feature with missing values (numeric) will be imputed using the iterative imputer.

In [1921]:
# Instantiating the IterativeImputer:
imputer = IterativeImputer(initial_strategy='mean', random_state = 42, max_iter = 20)

# Imputing the missing values:
non_applicable_data = imputer.fit_transform(train)
non_applicable_imputed = pd.DataFrame(non_applicable_data, columns = train.columns)

Saving the new data set for further analysis in the `model_training_and_evaluation` notebook. 

In [1922]:
# Saving the non-applicable dataset to pickle:
# non_applicable_imputed.to_pickle('../Data/Preprocessed/non_applicable_imputed.pkl')

## Streamlining for Data Preprocessing

Since the previous data preprocessing was quite substantial, a function will be defined to streamline this process for the test data set and in the eventuality if more data becomes available. The last method (that is, 'Creating an Extra Category for 'Non-Applicable') will be used to create this function, since it was the method with the best accuracy on the test set (see `model_training_and_evaluation` for more details).

In [102]:
def data_preprocessing(data):

    # Checking for duplicate entries:
    print(f'\nData is duplicated:', data.duplicated().any())

    # Checking whether the `id` column has only unique values:
    print(f'\nLength of unique ID values:', len(data['id'].unique()))

    # Convert 'Gender' to categorical:
    data['Gender'] = data['Gender'].astype('category')

    # Format 'Customer Type' column:
    data['Customer Type'] = data['Customer Type'].str.capitalize()
    data['Customer Type'] = data['Customer Type'].astype('category')

    # Validating the 'Age' column:
    print('\nMinimum Age:', data['Age'].min())
    print('Maximum Age:', data['Age'].max())

    # Format the 'Type of Travel' column:
    data['Type of Travel'] = data['Type of Travel'].str.capitalize()
    data['Type of Travel'] = data['Type of Travel'].astype('category')

    # Format the 'Class' column:
    data['Class'] = data['Class'].str.replace('Eco', '1')
    data['Class'] = data['Class'].str.replace('1 Plus', '2')
    data['Class'] = data['Class'].str.replace('Business', '3')
    data['Class'] = data['Class'].astype('category')
    data['Class'] = data['Class'].cat.set_categories(new_categories = ['1', '2', '3'], ordered = True)

    # Validate the 'Flight Distance' column:
    print('\nMinimum Flight Distance:', data['Flight Distance'].min())
    print('Maximum Flight Distance:', data['Flight Distance'].max())

    # Format the 'Inflight wifi service' column:
    data['Inflight wifi service'] = data['Inflight wifi service'].astype('category')
    data['Inflight wifi service'] = data['Inflight wifi service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Departure/Arrival time convenient' column:
    data['Departure/Arrival time convenient'] = data['Departure/Arrival time convenient'].astype('category')
    data['Departure/Arrival time convenient'] = data['Departure/Arrival time convenient'].cat.set_categories(new_categories = [1, 2, 3, 4, 5],
                                                                                                           ordered = True)
    
    # Format the 'Ease of Online booking' column:
    data['Ease of Online booking'] = data['Ease of Online booking'].astype('category')
    data['Ease of Online booking'] = data['Ease of Online booking'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], 
                                                                                     ordered = True)
    
    # Format the 'Gate location' column:
    data['Gate location'] = data['Gate location'].astype('category')
    data['Gate location'] = data['Gate location'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Food and drink' column:
    data['Food and drink'] = data['Food and drink'].astype('category')
    data['Food and drink'] = data['Food and drink'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Online boarding' column:
    data['Online boarding'] = data['Online boarding'].astype('category')
    data['Online boarding'] = data['Online boarding'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Seat comfort' column:
    data['Seat comfort'] = data['Seat comfort'].astype('category')
    data['Seat comfort'] = data['Seat comfort'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Inflight entertainment' column:
    data['Inflight entertainment'] = data['Inflight entertainment'].astype('category')
    data['Inflight entertainment'] = data['Inflight entertainment'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'On-board service' column:
    data['On-board service'] = data['On-board service'].astype('category')
    data['On-board service'] = data['On-board service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Leg room service' column:
    data['Leg room service'] = data['Leg room service'].astype('category')
    data['Leg room service'] = data['Leg room service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Baggage handling' column:
    data['Baggage handling'] = data['Baggage handling'].astype('category')
    data['Baggage handling'] = data['Baggage handling'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Checkin service' column:
    data['Checkin service'] = data['Checkin service'].astype('category')
    data['Checkin service'] = data['Checkin service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Inflight service' column:
    data['Inflight service'] = data['Inflight service'].astype('category')
    data['Inflight service'] = data['Inflight service'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Format the 'Cleanliness' column:
    data['Cleanliness'] = data['Cleanliness'].astype('category')
    data['Cleanliness'] = data['Cleanliness'].cat.set_categories(new_categories = [1, 2, 3, 4, 5], ordered = True)

    # Validate the 'Departure Delay in Minutes' column:
    print('\nMinimum Departure Delay in Minutes:', data['Departure Delay in Minutes'].min())
    print('Maximum Departure Delay in Minutes:',data['Departure Delay in Minutes'].max())

    # Validate the 'Arrival Delay in Minutes' column:
    print('\nMinimum Arrival Delay in Minutes:', data['Arrival Delay in Minutes'].min())
    print('Maximum Arrival Delay in Minutes:', data['Arrival Delay in Minutes'].max())

    # One-hot encoding nominal features:
    data = pd.get_dummies(data, columns = ['Gender', 'Customer Type', 'Type of Travel'], prefix = 'Dummy', dtype = int)

    # Format the 'satisfaction' column:
    data = data.rename(columns = {'satisfaction': 'Satisfaction'})
    # Defining the variable transformation needed:
    change = {'satisfied': 2, 'neutral or dissatisfied': 1}
    # Performing the transformation and converting to integer:
    data['Satisfaction'] = data['Satisfaction'].replace(change).infer_objects(copy=False)
    data['Satisfaction'] = data['Satisfaction'].astype('int')
    # Converting to ordered categorical:
    data['Satisfaction'] = data['Satisfaction'].astype('category')
    data['Satisfaction'] = data['Satisfaction'].cat.set_categories(new_categories = [1, 2], ordered = True)

    # Format other column name:
    data = data.rename(columns = {'Inflight wifi service': 'Inflight Wifi Service', 'Departure/Arrival time convenient': 'Departure/Arrival Time Convenient',
                                'Ease of Online booking': 'Ease of Online Booking', 'Gate location': 'Gate Location', 
                                'Food and drink': 'Food and Drink', 'Online boarding': 'Online Boarding', 'Seat comfort': 'Seat Comfort',
                                'Inflight entertainment': 'Inflight Entertainment', 'On-board service': 'On-board Service', 'Leg room service': 'Leg Room Service',
                                'Baggage handling': 'Baggage Handling', 'Checkin service': 'Checkin Service', 'Inflight service': 'Inflight Service'})
    
    def resolve_nans(df):
        # Defining the categorical features where NaNs could be located:
        non_applicable_category = ['Inflight Wifi Service', 'Departure/Arrival Time Convenient', 'Ease of Online Booking', 'Gate Location', 
                                   'Food and Drink', 'Online Boarding', 'Seat Comfort', 'Inflight Entertainment', 'On-board Service', 
                                   'Leg Room Service', 'Baggage Handling', 'Checkin Service', 'Inflight Service', 'Cleanliness']

        # Setting new category values for the defined features:
        for feature in non_applicable_category:
            df[feature] = df[feature].astype('category')
            df[feature] = df[feature].cat.set_categories(new_categories = [0, 1, 2, 3, 4, 5], ordered = True)
        
        # Replacing the NaNs with 0:
        assigning = {np.nan: 0}
        for feature in non_applicable_category:
            df[feature] = df[feature].fillna(0)
        
        # Instantiating the IterativeImputer:
        imputer = IterativeImputer(initial_strategy='mean', random_state = 42, max_iter = 20)

        # Imputing the missing values for numeric data:
        numeric_data = imputer.fit_transform(df)
        numeric_data_imputed = pd.DataFrame(numeric_data, columns = data.columns)

        return numeric_data_imputed
    
    data = resolve_nans(data)
    # Checking the shape of the dataset:
    print('\nData shape:', data.shape)
    
    return data



## Cleaning and Transforming the Test Data Set

The same process is performed on the test set.

In [104]:
# Importing the data set:
test = pd.read_csv('../Data/Raw/test.csv', index_col = 0)

In [105]:
# Using the created function to process the test set:
test = data_preprocessing(test)


Data is duplicated: False

Length of unique ID values: 25976

Minimum Age: 7
Maximum Age: 85

Minimum Flight Distance: 31
Maximum Flight Distance: 4983

Minimum Departure Delay in Minutes: 0
Maximum Departure Delay in Minutes: 1128

Minimum Arrival Delay in Minutes: 0.0
Maximum Arrival Delay in Minutes: 1115.0


/var/folders/1z/2qtxqpxj07g0pvc7h463fm_w0000gn/T/ipykernel_92608/3260739351.py:109: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Satisfaction'] = data['Satisfaction'].replace(change).infer_objects(copy=False)



Data shape: (25976, 27)


Both datasets are now ready. The next step is to perform an Exploratory Data Analysis on the train set. Please go to the `exploratory_data_analysis` notebook. 

In [107]:
# Saving the preprocessed test dataset to pickle:
# test.to_pickle('../Data/Preprocessed/test_preprocessed.pkl')

## References

1. <a id="ref-scikitdevelopers2024"></a>Scikit-learn developers, IterativeImputer, *Scikit-Learn*, 2024. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html)
2. <a id="ref-htoon2020"></a>Kyaw Saw Htoon, A Guide to KNN Imputation, *Medium*, 3 July 2020. [Link](https://medium.com/@kyawsawhtoon/a-guide-to-knn-imputation-95e2dc496e)
3. <a id="ref-Igkobe2024"></a>Hannah Igboke, Iterative Imputer for Missing values in Machine Learning, *Medium*, 10 June 2024. [Link](https://medium.com/learning-data/iterative-imputer-for-missing-values-in-machine-learning-32bd8b5b697a#:~:text=Statistical%20models%20used%20in%20iterative%20imputation&text=DecisionTreeRegressor%3A%20non%2Dlinear%20regression%20models,no%20need%20for%20feature%20scaling.)